In [1]:
import cv2
from ultralytics import YOLO
from datetime import datetime
import numpy as np

class PersonTracker:
    def __init__(self, max_people=10, track_history_length=30):
        # Load YOLO model
        self.model = YOLO('yolov8n.pt')
        
        # Tracking variables
        self.track_history = {}
        self.max_people = max_people
        self.track_history_length = track_history_length
    
    def update_tracks(self, frame, results):
        """
        Update tracking information for detected persons
        """
        current_tracks = {}
        
        # Process each detected person
        for box in results[0].boxes:
            # Get bounding box coordinates
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = float(box.conf[0])
            
            # Find or create track
            track_id = self.get_track_id(box)
            
            # Update track history
            if track_id not in self.track_history:
                self.track_history[track_id] = []
            
            # Add current position to track history
            self.track_history[track_id].append((x1, y1, x2, y2))
            
            # Limit track history length
            if len(self.track_history[track_id]) > self.track_history_length:
                self.track_history[track_id].pop(0)
            
            current_tracks[track_id] = {
                'bbox': (x1, y1, x2, y2),
                'confidence': conf
            }
        
        return current_tracks
    
    def get_track_id(self, box):
        """
        Generate or match track ID based on IoU (Intersection over Union)
        """
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        
        # If no existing tracks, create a new ID
        if not self.track_history:
            return 0
        
        # Check for potential matches with existing tracks
        for track_id, history in self.track_history.items():
            if history:
                last_bbox = history[-1]
                iou = self.calculate_iou(
                    (x1, y1, x2, y2), 
                    last_bbox
                )
                
                # If IoU is high, consider it the same track
                if iou > 0.5:
                    return track_id
        
        # If no match, create new track ID
        return max(self.track_history.keys()) + 1 if self.track_history else 0
    
    def calculate_iou(self, box1, box2):
        """
        Calculate Intersection over Union between two bounding boxes
        """
        x1_1, y1_1, x2_1, y2_1 = box1
        x1_2, y1_2, x2_2, y2_2 = box2
        
        # Calculate intersection coordinates
        x1_inter = max(x1_1, x1_2)
        y1_inter = max(y1_1, y1_2)
        x2_inter = min(x2_1, x2_2)
        y2_inter = min(y2_1, y2_2)
        
        # Calculate intersection area
        inter_area = max(0, x2_inter - x1_inter) * max(0, y2_inter - y1_inter)
        
        # Calculate union area
        box1_area = (x2_1 - x1_1) * (y2_1 - y1_1)
        box2_area = (x2_2 - x1_2) * (y2_2 - y1_2)
        union_area = box1_area + box2_area - inter_area
        
        # Calculate IoU
        return inter_area / union_area if union_area > 0 else 0

def count_people(video_path, output_path=None):
    """
    Process a video file, detect, track, and count people
    """
    # Initialize tracker
    tracker = PersonTracker()

    # Read video
    cap = cv2.VideoCapture(video_path)

    # Get video properties
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    # Setup video writer if output path is provided
    if output_path:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1
        print(f"Processing frame {frame_count}")

        # Run YOLOv8 inference on the frame
        results = tracker.model(frame, classes=[0])  # class 0 is person in COCO

        # Update and get current tracks
        current_tracks = tracker.update_tracks(frame, results)

        # Draw tracked persons
        for track_id, track_info in current_tracks.items():
            x1, y1, x2, y2 = track_info['bbox']
            conf = track_info['confidence']

            # Draw bounding box
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            
            # Draw track ID and confidence
            cv2.putText(frame, 
                        f"ID: {track_id} ({conf:.2f})", 
                        (x1, y1-10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 
                        0.5, 
                        (0, 255, 0), 
                        2)

        # Display people count
        people_count = len(current_tracks)
        cv2.putText(
            frame,
            f"People Count: {people_count}",
            (20, 40),
            cv2.FONT_HERSHEY_SIMPLEX,
            1,
            (0, 255, 0),
            2
        )

        # Alert for too many people
        if people_count > tracker.max_people:
            cv2.putText(
                frame,
                "ALERT: Too many people!",
                (20, 80),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (0, 0, 255),
                2
            )
            print(f"[{datetime.now()}] Alert: {people_count} people detected (max: {tracker.max_people})")

        # Save and display frame
        if output_path:
            out.write(frame)

        cv2.imshow('People Tracking', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Cleanup
    cap.release()
    if output_path:
        out.release()
    cv2.destroyAllWindows()

def main():
    video_path = input("Enter the video file path: ")
    output_path = input("Enter the output file path (or press Enter to skip saving): ")

    count_people(video_path, output_path if output_path else None)

if __name__ == "__main__":
    main()

Processing frame 1

0: 384x640 10 persons, 155.1ms
Speed: 6.0ms preprocess, 155.1ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)
Processing frame 2

0: 384x640 11 persons, 125.2ms
Speed: 5.0ms preprocess, 125.2ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)
Processing frame 3

0: 384x640 12 persons, 189.6ms
Speed: 3.0ms preprocess, 189.6ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)
Processing frame 4

0: 384x640 12 persons, 111.2ms
Speed: 3.0ms preprocess, 111.2ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
Processing frame 5

0: 384x640 12 persons, 97.3ms
Speed: 4.0ms preprocess, 97.3ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
Processing frame 6

0: 384x640 10 persons, 110.2ms
Speed: 2.0ms preprocess, 110.2ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
Processing frame 7

0: 384x640 11 persons, 107.2ms
Speed: 3.0ms preprocess, 107.2ms inference, 3.0ms postpro

In [2]:
import cv2
from ultralytics import YOLO
import numpy as np

class PersonTracker:
    def __init__(self, track_timeout=30):
        # Load YOLO model for detection
        self.model = YOLO('yolov8n.pt')
        
        # Tracking variables
        self.active_tracks = {}
        self.track_id_counter = 0
        self.track_timeout = track_timeout
        self.frame_count_since_last_detection = {}

    def track_people(self, frame):
        """
        Detect and track people in the frame
        """
        # Detect all objects
        results = self.model(frame)
        
        # Filter only human detections
        human_detections = []
        for result in results:
            boxes = result.boxes
            for box in boxes:
                # Check if the detected object is a person (class 0 in COCO dataset)
                if int(box.cls[0]) == 0:
                    human_detections.append(box)
        
        # Update existing tracks and create new ones
        current_tracks = {}
        for detection in human_detections:
            x1, y1, x2, y2 = map(int, detection.xyxy[0])
            confidence = float(detection.conf[0])
            
            # Try to match with existing tracks
            matched = False
            for track_id, track_info in list(self.active_tracks.items()):
                # Calculate IoU to match tracks
                iou = self.calculate_iou(
                    (x1, y1, x2, y2), 
                    track_info['last_bbox']
                )
                
                # If IoU is high, update existing track
                if iou > 0.3:
                    self.active_tracks[track_id].update({
                        'last_bbox': (x1, y1, x2, y2),
                        'last_seen': 0,
                        'confidence': confidence
                    })
                    current_tracks[track_id] = self.active_tracks[track_id]
                    matched = True
                    break
            
            # If no match, create a new track
            if not matched:
                new_track_id = self.track_id_counter
                self.track_id_counter += 1
                current_tracks[new_track_id] = {
                    'last_bbox': (x1, y1, x2, y2),
                    'last_seen': 0,
                    'confidence': confidence
                }
                self.active_tracks[new_track_id] = current_tracks[new_track_id]
        
        # Remove tracks that haven't been seen for too long
        for track_id in list(self.active_tracks.keys()):
            self.active_tracks[track_id]['last_seen'] += 1
            if self.active_tracks[track_id]['last_seen'] > self.track_timeout:
                del self.active_tracks[track_id]
        
        return current_tracks

    def calculate_iou(self, box1, box2):
        """
        Calculate Intersection over Union between two bounding boxes
        """
        x1_1, y1_1, x2_1, y2_1 = box1
        x1_2, y1_2, x2_2, y2_2 = box2
        
        # Calculate intersection coordinates
        x1_inter = max(x1_1, x1_2)
        y1_inter = max(y1_1, y1_2)
        x2_inter = min(x2_1, x2_2)
        y2_inter = min(y2_1, y2_2)
        
        # Calculate intersection area
        inter_area = max(0, x2_inter - x1_inter) * max(0, y2_inter - y1_inter)
        
        # Calculate union area
        box1_area = (x2_1 - x1_1) * (y2_1 - y1_1)
        box2_area = (x2_2 - x1_2) * (y2_2 - y1_2)
        union_area = box1_area + box2_area - inter_area
        
        # Calculate IoU
        return inter_area / union_area if union_area > 0 else 0

def process_video(video_path, output_path=None):
    """
    Process video for people tracking and counting
    """
    # Initialize tracker
    tracker = PersonTracker()

    # Open video capture
    cap = cv2.VideoCapture(video_path)

    # Get video properties
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    # Setup video writer if output path is provided
    if output_path:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1

        # Track people in the frame
        active_tracks = tracker.track_people(frame)

        # Draw bounding boxes and track information
        for track_id, track_info in active_tracks.items():
            x1, y1, x2, y2 = track_info['last_bbox']
            conf = track_info['confidence']

            # Draw bounding box
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            
            # Draw track ID and confidence
            cv2.putText(frame, 
                        f"ID: {track_id} (Conf: {conf:.2f})", 
                        (x1, y1-10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 
                        0.5, 
                        (0, 255, 0), 
                        2)

        # Display people count
        people_count = len(active_tracks)
        cv2.putText(
            frame,
            f"People Count: {people_count}",
            (20, 40),
            cv2.FONT_HERSHEY_SIMPLEX,
            1,
            (0, 255, 0),
            2
        )

        # Save and display frame
        if output_path:
            out.write(frame)

        cv2.imshow('People Tracking', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Cleanup
    cap.release()
    if output_path:
        out.release()
    cv2.destroyAllWindows()

def main():
    video_path = input("Enter the video file path: ")
    output_path = input("Enter the output file path (or press Enter to skip saving): ")

    process_video(video_path, output_path if output_path else None)

if __name__ == "__main__":
    main()


0: 384x640 10 persons, 7 cars, 4 trucks, 122.2ms
Speed: 3.0ms preprocess, 122.2ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 persons, 6 cars, 5 trucks, 141.7ms
Speed: 2.0ms preprocess, 141.7ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 persons, 5 cars, 6 trucks, 130.2ms
Speed: 4.0ms preprocess, 130.2ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 persons, 6 cars, 4 trucks, 125.5ms
Speed: 2.0ms preprocess, 125.5ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 persons, 8 cars, 4 trucks, 155.1ms
Speed: 4.0ms preprocess, 155.1ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 persons, 8 cars, 4 trucks, 120.2ms
Speed: 3.0ms preprocess, 120.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 persons, 7 cars, 4 trucks, 116.9ms
Speed: 3.0ms preprocess, 116.9ms inference, 3.0ms postprocess 

In [3]:
#2
import cv2
from ultralytics import YOLO
import numpy as np

class PersonTracker:
    def __init__(self, track_timeout=30):
        # Load YOLO model for detection
        self.model = YOLO('yolov8n.pt')
        
        # Tracking variables
        self.active_tracks = {}
        self.track_id_counter = 0
        self.track_timeout = track_timeout
        self.total_unique_persons = set()

    def track_people(self, frame):
        """
        Detect and track people in the frame
        """
        # Detect all objects
        results = self.model(frame)
        
        # Filter only human detections with high confidence
        human_detections = []
        for result in results:
            boxes = result.boxes
            for box in boxes:
                # Strict filtering: person class and high confidence
                if (int(box.cls[0]) == 0 and  # Person class
                    float(box.conf[0]) > 0.7):  # High confidence threshold
                    human_detections.append(box)
        
        # Update existing tracks and create new ones
        current_tracks = {}
        for detection in human_detections:
            x1, y1, x2, y2 = map(int, detection.xyxy[0])
            confidence = float(detection.conf[0])
            
            # Try to match with existing tracks
            matched = False
            for track_id, track_info in list(self.active_tracks.items()):
                # Calculate IoU to match tracks
                iou = self.calculate_iou(
                    (x1, y1, x2, y2), 
                    track_info['last_bbox']
                )
                
                # If IoU is high, update existing track
                if iou > 0.3:
                    self.active_tracks[track_id].update({
                        'last_bbox': (x1, y1, x2, y2),
                        'last_seen': 0,
                        'confidence': confidence
                    })
                    current_tracks[track_id] = self.active_tracks[track_id]
                    matched = True
                    break
            
            # If no match, create a new track
            if not matched:
                new_track_id = self.track_id_counter
                self.track_id_counter += 1
                current_tracks[new_track_id] = {
                    'last_bbox': (x1, y1, x2, y2),
                    'last_seen': 0,
                    'confidence': confidence
                }
                self.active_tracks[new_track_id] = current_tracks[new_track_id]
                # Track unique persons
                self.total_unique_persons.add(new_track_id)
        
        # Remove tracks that haven't been seen for too long
        for track_id in list(self.active_tracks.keys()):
            self.active_tracks[track_id]['last_seen'] += 1
            if self.active_tracks[track_id]['last_seen'] > self.track_timeout:
                del self.active_tracks[track_id]
        
        return current_tracks

    def calculate_iou(self, box1, box2):
        """
        Calculate Intersection over Union between two bounding boxes
        """
        x1_1, y1_1, x2_1, y2_1 = box1
        x1_2, y1_2, x2_2, y2_2 = box2
        
        # Calculate intersection coordinates
        x1_inter = max(x1_1, x1_2)
        y1_inter = max(y1_1, y1_2)
        x2_inter = min(x2_1, x2_2)
        y2_inter = min(y2_1, y2_2)
        
        # Calculate intersection area
        inter_area = max(0, x2_inter - x1_inter) * max(0, y2_inter - y1_inter)
        
        # Calculate union area
        box1_area = (x2_1 - x1_1) * (y2_1 - y1_1)
        box2_area = (x2_2 - x1_2) * (y2_2 - y1_2)
        union_area = box1_area + box2_area - inter_area
        
        # Calculate IoU
        return inter_area / union_area if union_area > 0 else 0

def process_video(video_path, output_path=None):
    """
    Process video for people tracking and counting
    """
    # Initialize tracker
    tracker = PersonTracker()

    # Open video capture
    cap = cv2.VideoCapture(video_path)

    # Get video properties
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    # Setup video writer if output path is provided
    if output_path:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1

        # Track people in the frame
        active_tracks = tracker.track_people(frame)

        # Prepare text for display
        info_text = [
            f"Current People: {len(active_tracks)}",
            f"Total Unique People: {len(tracker.total_unique_persons)}"
        ]

        # Display tracking information
        for y, text in enumerate(info_text):
            cv2.putText(
                frame,
                text,
                (20, 40 + y*40),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (0, 255, 0),
                2
            )

        # Save and display frame
        if output_path:
            out.write(frame)

        cv2.imshow('People Tracking', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Cleanup
    cap.release()
    if output_path:
        out.release()
    cv2.destroyAllWindows()

    # Final summary
    print(f"Total unique people tracked: {len(tracker.total_unique_persons)}")

def main():
    video_path = input("Enter the video file path: ")
    output_path = input("Enter the output file path (or press Enter to skip saving): ")

    process_video(video_path, output_path if output_path else None)

if __name__ == "__main__":
    main()


0: 384x640 10 persons, 7 cars, 4 trucks, 131.7ms
Speed: 3.0ms preprocess, 131.7ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 persons, 6 cars, 5 trucks, 158.7ms
Speed: 4.0ms preprocess, 158.7ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 persons, 5 cars, 6 trucks, 135.2ms
Speed: 3.0ms preprocess, 135.2ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 persons, 6 cars, 4 trucks, 142.2ms
Speed: 3.5ms preprocess, 142.2ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 persons, 8 cars, 4 trucks, 129.7ms
Speed: 4.0ms preprocess, 129.7ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 persons, 8 cars, 4 trucks, 110.2ms
Speed: 4.0ms preprocess, 110.2ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 persons, 7 cars, 4 trucks, 125.2ms
Speed: 2.0ms preprocess, 125.2ms inference, 3.0ms postprocess 

In [7]:
import cv2
from ultralytics import YOLO
import numpy as np

class PersonTracker:
    def __init__(self, track_timeout=30):
        # Load YOLO model for detection
        self.model = YOLO('F:\insighteye\datacenter\yolov8l.pt')
        
        # Tracking variables
        self.active_tracks = {}
        self.track_id_counter = 0
        self.track_timeout = track_timeout
        self.total_unique_persons = set()

    def track_people(self, frame):
        """
        Detect and track people in the frame
        """
        # Detect all objects
        results = self.model(frame)
        
        # Filter only human detections with high confidence
        human_detections = []
        for result in results:
            boxes = result.boxes
            for box in boxes:
                # Strict filtering: person class and high confidence
                if (int(box.cls[0]) == 0 and  # Person class
                    float(box.conf[0]) > 0.7):  # High confidence threshold
                    human_detections.append(box)
        
        # Update existing tracks and create new ones
        current_tracks = {}
        for detection in human_detections:
            x1, y1, x2, y2 = map(int, detection.xyxy[0])
            confidence = float(detection.conf[0])
            
            # Try to match with existing tracks
            matched = False
            for track_id, track_info in list(self.active_tracks.items()):
                # Calculate IoU to match tracks
                iou = self.calculate_iou(
                    (x1, y1, x2, y2), 
                    track_info['last_bbox']
                )
                
                # If IoU is high, update existing track
                if iou > 0.3:
                    self.active_tracks[track_id].update({
                        'last_bbox': (x1, y1, x2, y2),
                        'last_seen': 0,
                        'confidence': confidence
                    })
                    current_tracks[track_id] = self.active_tracks[track_id]
                    matched = True
                    break
            
            # If no match, create a new track
            if not matched:
                new_track_id = self.track_id_counter
                self.track_id_counter += 1
                current_tracks[new_track_id] = {
                    'last_bbox': (x1, y1, x2, y2),
                    'last_seen': 0,
                    'confidence': confidence
                }
                self.active_tracks[new_track_id] = current_tracks[new_track_id]
                # Track unique persons
                self.total_unique_persons.add(new_track_id)
        
        # Remove tracks that haven't been seen for too long
        for track_id in list(self.active_tracks.keys()):
            self.active_tracks[track_id]['last_seen'] += 1
            if self.active_tracks[track_id]['last_seen'] > self.track_timeout:
                del self.active_tracks[track_id]
        
        return current_tracks

    def calculate_iou(self, box1, box2):
        """
        Calculate Intersection over Union between two bounding boxes
        """
        x1_1, y1_1, x2_1, y2_1 = box1
        x1_2, y1_2, x2_2, y2_2 = box2
        
        # Calculate intersection coordinates
        x1_inter = max(x1_1, x1_2)
        y1_inter = max(y1_1, y1_2)
        x2_inter = min(x2_1, x2_2)
        y2_inter = min(y2_1, y2_2)
        
        # Calculate intersection area
        inter_area = max(0, x2_inter - x1_inter) * max(0, y2_inter - y1_inter)
        
        # Calculate union area
        box1_area = (x2_1 - x1_1) * (y2_1 - y1_1)
        box2_area = (x2_2 - x1_2) * (y2_2 - y1_2)
        union_area = box1_area + box2_area - inter_area
        
        # Calculate IoU
        return inter_area / union_area if union_area > 0 else 0

def process_video(video_path, output_path=None):
    """
    Process video for people tracking and counting
    """
    # Video path can be hardcoded here
    VIDEO_PATH = video_path
    
    # Initialize tracker
    tracker = PersonTracker()

    # Open video capture
    cap = cv2.VideoCapture(VIDEO_PATH)

    # Get video properties
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    # Setup video writer if output path is provided
    if output_path:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1

        # Track people in the frame
        active_tracks = tracker.track_people(frame)

        # Draw bounding boxes and labels for each tracked person
        for track_id, track_info in active_tracks.items():
            x1, y1, x2, y2 = track_info['last_bbox']
            
            # Draw thin bounding box
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 1)
            
            # Create label with track ID and confidence
            label = f"ID: {track_id} ({track_info['confidence']:.2f})"
            
            # Put label above the bounding box
            cv2.putText(frame, label, (x1, y1-10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

        # Prepare text for display
        info_text = [
            f"Current People: {len(active_tracks)}",
            f"Total Unique People: {len(tracker.total_unique_persons)}"
        ]

        # Display tracking information
        for y, text in enumerate(info_text):
            cv2.putText(
                frame,
                text,
                (20, 40 + y*40),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (0, 255, 0),
                2
            )

        # Save and display frame
        if output_path:
            out.write(frame)

        cv2.imshow('People Tracking', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Cleanup
    cap.release()
    if output_path:
        out.release()
    cv2.destroyAllWindows()

    # Final summary
    print(f"Total unique people tracked: {len(tracker.total_unique_persons)}")

def main():
    # Hardcoded video path for easy modification
    VIDEO_PATH = 'F:\insighteye\datacenter\people.mp4'
    OUTPUT_PATH = 'F:\insighteye\datacenter\people Using-YOLOv8-and-DeepSORT-main\people_output.mp4'

    process_video(VIDEO_PATH, OUTPUT_PATH)

if __name__ == "__main__":
    main()

<>:8: SyntaxWarning: invalid escape sequence '\i'
<>:190: SyntaxWarning: invalid escape sequence '\i'
<>:191: SyntaxWarning: invalid escape sequence '\i'
<>:8: SyntaxWarning: invalid escape sequence '\i'
<>:190: SyntaxWarning: invalid escape sequence '\i'
<>:191: SyntaxWarning: invalid escape sequence '\i'
C:\Users\Bobby\AppData\Local\Temp\ipykernel_15136\1026352014.py:8: SyntaxWarning: invalid escape sequence '\i'
  self.model = YOLO('F:\insighteye\datacenter\yolov8l.pt')
C:\Users\Bobby\AppData\Local\Temp\ipykernel_15136\1026352014.py:190: SyntaxWarning: invalid escape sequence '\i'
  VIDEO_PATH = 'F:\insighteye\datacenter\people.mp4'
C:\Users\Bobby\AppData\Local\Temp\ipykernel_15136\1026352014.py:191: SyntaxWarning: invalid escape sequence '\i'
  OUTPUT_PATH = 'F:\insighteye\datacenter\people Using-YOLOv8-and-DeepSORT-main\people_output.mp4'



0: 384x640 29 persons, 3 handbags, 1 frisbee, 828.1ms
Speed: 3.0ms preprocess, 828.1ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 29 persons, 3 handbags, 764.4ms
Speed: 4.0ms preprocess, 764.4ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 29 persons, 3 handbags, 730.6ms
Speed: 3.0ms preprocess, 730.6ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 28 persons, 1 handbag, 717.6ms
Speed: 2.0ms preprocess, 717.6ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 29 persons, 1 handbag, 689.6ms
Speed: 3.0ms preprocess, 689.6ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 30 persons, 3 handbags, 1 frisbee, 727.3ms
Speed: 4.0ms preprocess, 727.3ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 29 persons, 3 handbags, 696.5ms
Speed: 2.0ms preprocess, 696.5ms inference, 3.0ms postprocess per image at shape (1,

In [9]:
import cv2
import torch
import numpy as np

# Load the YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# Set the video source (0 for webcam, or provide a video file path)
video_source = 'F:\insighteye\datacenter\people Using-YOLOv8-and-DeepSORT-main\people_animal.mp4'
cap = cv2.VideoCapture(video_source)

# Define the output video writer (optional)
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
out = cv2.VideoWriter('F:\insighteye\datacenter\peepani.mp4', cv2.VideoWriter_fourcc(*'XVID'), 30, (frame_width, frame_height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Perform inference on the frame
    results = model(frame)
    
    # Extract detection results
    detections = results.xyxy[0].cpu().numpy()
    
    person_count = 0
    for *xyxy, conf, cls in detections:
        # Filter for 'person' class (class ID 0 in COCO dataset)
        if int(cls) == 0:
            person_count += 1
            # Draw bounding box and label
            x1, y1, x2, y2 = map(int, xyxy)
            label = f'Person {conf:.2f}'
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Display the person count on the frame
    cv2.putText(frame, f'Persons Detected: {person_count}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Show the video frame
    cv2.imshow('Person Detection and Counter', frame)

    # Write the frame to output video (optional)
    out.write(frame)

    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()

<>:9: SyntaxWarning: invalid escape sequence '\i'
<>:15: SyntaxWarning: invalid escape sequence '\i'
<>:9: SyntaxWarning: invalid escape sequence '\i'
<>:15: SyntaxWarning: invalid escape sequence '\i'
C:\Users\Bobby\AppData\Local\Temp\ipykernel_15136\2318645624.py:9: SyntaxWarning: invalid escape sequence '\i'
  video_source = 'F:\insighteye\datacenter\people Using-YOLOv8-and-DeepSORT-main\people_animal.mp4'
C:\Users\Bobby\AppData\Local\Temp\ipykernel_15136\2318645624.py:15: SyntaxWarning: invalid escape sequence '\i'
  out = cv2.VideoWriter('F:\insighteye\datacenter\peepani.mp4', cv2.VideoWriter_fourcc(*'XVID'), 30, (frame_width, frame_height))
Using cache found in C:\Users\Bobby/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2022-1-14 torch 2.2.1+cpu CPU

Fusing layers... 
Model Summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [13]:
#person and animal 
import cv2
import torch
import numpy as np

# Load the YOLOv8 model (person and animal detection)
model = torch.hub.load('ultralytics/yolov8', 'yolov8m', pretrained=True)

# Set the video source (0 for webcam, or provide a video file path)

video_source = 'F:\insighteye\datacenter\people Using-YOLOv8-and-DeepSORT-main\people_animal.mp4'
cap = cv2.VideoCapture(video_source)

# Define the output video writer (optional)
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
out = cv2.VideoWriter('F:\insighteye\datacenter\pplnani.mp4', cv2.VideoWriter_fourcc(*'XVID'), 30, (frame_width, frame_height))

# Define colors for classes
colors = {
    'person': (0, 255, 0),  # Green for person
    'animal': (255, 0, 0)   # Blue for animals
}

# Class IDs for person and animal in COCO dataset
person_class_id = 0
animal_class_ids = [16, 17, 18, 19, 20, 21 , 22]  # Animal classes: bird, cat, dog, horse, sheep, cow, elephant

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Perform inference on the frame
    results = model(frame)
    
    # Extract detection results
    detections = results.xyxy[0].cpu().numpy()
    
    person_count = 0
    animal_count = 0

    for *xyxy, conf, cls in detections:
        cls = int(cls)
        x1, y1, x2, y2 = map(int, xyxy)
        
        # Check for 'person' class
        if cls == person_class_id:
            person_count += 1
            label = f'Person {conf:.2f}'
            color = colors['person']
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
        
        # Check for 'animal' classes
        elif cls in animal_class_ids:
            animal_count += 1
            label = f'Animal {conf:.2f}'
            color = colors['animal']
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Display the counts on the frame
    cv2.putText(frame, f'Persons Detected: {person_count}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.putText(frame, f'Animals Detected: {animal_count}', (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    # Show the video frame
    cv2.imshow('Person and Animal Detection', frame)

    # Write the frame to output video (optional)
    out.write(frame)

    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()


<>:11: SyntaxWarning: invalid escape sequence '\i'
<>:17: SyntaxWarning: invalid escape sequence '\i'
<>:11: SyntaxWarning: invalid escape sequence '\i'
<>:17: SyntaxWarning: invalid escape sequence '\i'
C:\Users\Bobby\AppData\Local\Temp\ipykernel_15136\427004995.py:11: SyntaxWarning: invalid escape sequence '\i'
  video_source = 'F:\insighteye\datacenter\people Using-YOLOv8-and-DeepSORT-main\people_animal.mp4'
C:\Users\Bobby\AppData\Local\Temp\ipykernel_15136\427004995.py:17: SyntaxWarning: invalid escape sequence '\i'
  out = cv2.VideoWriter('F:\insighteye\datacenter\pplnani.mp4', cv2.VideoWriter_fourcc(*'XVID'), 30, (frame_width, frame_height))
C:\Users\Bobby\AppData\Local\Temp\ipykernel_15136\427004995.py:11: SyntaxWarning: invalid escape sequence '\i'
  video_source = 'F:\insighteye\datacenter\people Using-YOLOv8-and-DeepSORT-main\people_animal.mp4'
C:\Users\Bobby\AppData\Local\Temp\ipykernel_15136\427004995.py:17: SyntaxWarning: invalid escape sequence '\i'
  out = cv2.VideoWrite

HTTPError: HTTP Error 404: Not Found

In [14]:
import cv2
import numpy as np
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO('F:\insighteye\datacenter\yolov8l.pt')  # Replace with 'yolov8m.pt' for medium model

# Set the video source (0 for webcam, or provide a video file path)
video_source = 'F:\insighteye\datacenter\people Using-YOLOv8-and-DeepSORT-main\people_animal.mp4'  # Update to video path if needed
cap = cv2.VideoCapture(video_source)

# Define the output video writer (optional)
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
out = cv2.VideoWriter('F:\insighteye\datacenter\pplnani.mp4', cv2.VideoWriter_fourcc(*'XVID'), 30, (frame_width, frame_height))

# Define colors for classes
colors = {
    'person': (0, 255, 0),  # Green for person
    'animal': (255, 0, 0)   # Blue for animals
}

# Class IDs for COCO dataset
person_class_id = 0  # Person class
animal_class_ids = [16, 17, 18, 19, 20, 21, 22]  # Bird, cat, dog, horse, sheep, cow, elephant

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Perform inference on the frame
    results = model(frame)

    person_count = 0
    animal_count = 0

    for result in results:
        for box in result.boxes.data:
            x1, y1, x2, y2, conf, cls = map(int, box[:6])
            cls = int(cls)

            # Check for 'person' class
            if cls == person_class_id:
                person_count += 1
                label = f'Person {conf:.2f}'
                color = colors['person']
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

            # Check for 'animal' classes
            elif cls in animal_class_ids:
                animal_count += 1
                label = f'Animal {conf:.2f}'
                color = colors['animal']
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Display the counts on the frame
    cv2.putText(frame, f'Persons Detected: {person_count}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.putText(frame, f'Animals Detected: {animal_count}', (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    # Show the video frame
    cv2.imshow('Person and Animal Detection', frame)

    # Write the frame to output video (optional)
    out.write(frame)

    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()


<>:6: SyntaxWarning: invalid escape sequence '\i'
<>:9: SyntaxWarning: invalid escape sequence '\i'
<>:15: SyntaxWarning: invalid escape sequence '\i'
<>:6: SyntaxWarning: invalid escape sequence '\i'
<>:9: SyntaxWarning: invalid escape sequence '\i'
<>:15: SyntaxWarning: invalid escape sequence '\i'
C:\Users\Bobby\AppData\Local\Temp\ipykernel_15136\852439833.py:6: SyntaxWarning: invalid escape sequence '\i'
  model = YOLO('F:\insighteye\datacenter\yolov8l.pt')  # Replace with 'yolov8m.pt' for medium model
C:\Users\Bobby\AppData\Local\Temp\ipykernel_15136\852439833.py:9: SyntaxWarning: invalid escape sequence '\i'
  video_source = 'F:\insighteye\datacenter\people Using-YOLOv8-and-DeepSORT-main\people_animal.mp4'  # Update to video path if needed
C:\Users\Bobby\AppData\Local\Temp\ipykernel_15136\852439833.py:15: SyntaxWarning: invalid escape sequence '\i'
  out = cv2.VideoWriter('F:\insighteye\datacenter\pplnani.mp4', cv2.VideoWriter_fourcc(*'XVID'), 30, (frame_width, frame_height))



0: 384x640 14 persons, 7 cars, 1 motorcycle, 4 trucks, 2 elephants, 837.0ms
Speed: 2.0ms preprocess, 837.0ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 14 persons, 8 cars, 2 motorcycles, 3 trucks, 2 elephants, 754.1ms
Speed: 4.0ms preprocess, 754.1ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 persons, 7 cars, 2 motorcycles, 2 trucks, 2 elephants, 741.7ms
Speed: 3.0ms preprocess, 741.7ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 persons, 10 cars, 2 motorcycles, 2 trucks, 3 elephants, 733.7ms
Speed: 2.5ms preprocess, 733.7ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 14 persons, 8 cars, 2 motorcycles, 3 trucks, 3 elephants, 777.1ms
Speed: 3.0ms preprocess, 777.1ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 14 persons, 9 cars, 2 motorcycles, 3 trucks, 3 elephants, 730.7ms
Speed: 2.0ms preprocess, 730.7ms inference

In [15]:
import cv2
import numpy as np
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO('F:\insighteye\datacenter\yolov8l.pt')  # Replace with 'yolov8m.pt' for medium model


# Set the video source (0 for webcam, or provide a video file path)
video_source = 'F:\insighteye\datacenter\people Using-YOLOv8-and-DeepSORT-main\people_animal.mp4'  # Update to video path if needed
cap = cv2.VideoCapture(video_source)

# Define the output video writer (optional)
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
out = cv2.VideoWriter('F:\insighteye\datacenter\peopleNanimal.mp4', cv2.VideoWriter_fourcc(*'XVID'), 30, (frame_width, frame_height))

# Define colors for classes
colors = {
    'person': (0, 255, 0),  # Green for person
    'animal': (255, 0, 0)   # Blue for animals
}

# Class IDs for COCO dataset
person_class_id = 0  # Person class
animal_class_ids = [16, 17, 18, 19, 20, 21, 22]  # Bird, cat, dog, horse, sheep, cow, elephant

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Perform inference on the frame
    results = model(frame)

    person_count = 0
    animal_count = 0

    for result in results:
        for box in result.boxes.data:
            x1, y1, x2, y2 = map(int, box[:4])  # Bounding box coordinates
            conf = float(box[4])                # Confidence score (as float)
            cls = int(box[5])                   # Class ID (as integer)

            # Check for 'person' class
            if cls == person_class_id:
                person_count += 1
                label = f'Person {conf:.2f}'  # Display confidence up to 2 decimals
                color = colors['person']
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

            # Check for 'animal' classes
            elif cls in animal_class_ids:
                animal_count += 1
                label = f'Animal {conf:.2f}'  # Display confidence up to 2 decimals
                color = colors['animal']
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Display the counts on the frame
    cv2.putText(frame, f'Persons Detected: {person_count}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.putText(frame, f'Animals Detected: {animal_count}', (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    # Show the video frame
    cv2.imshow('Person and Animal Detection', frame)

    # Write the frame to output video (optional)
    out.write(frame)

    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()


<>:6: SyntaxWarning: invalid escape sequence '\i'
<>:10: SyntaxWarning: invalid escape sequence '\i'
<>:16: SyntaxWarning: invalid escape sequence '\i'
<>:6: SyntaxWarning: invalid escape sequence '\i'
<>:10: SyntaxWarning: invalid escape sequence '\i'
<>:16: SyntaxWarning: invalid escape sequence '\i'
C:\Users\Bobby\AppData\Local\Temp\ipykernel_15136\2079917165.py:6: SyntaxWarning: invalid escape sequence '\i'
  model = YOLO('F:\insighteye\datacenter\yolov8l.pt')  # Replace with 'yolov8m.pt' for medium model
C:\Users\Bobby\AppData\Local\Temp\ipykernel_15136\2079917165.py:10: SyntaxWarning: invalid escape sequence '\i'
  video_source = 'F:\insighteye\datacenter\people Using-YOLOv8-and-DeepSORT-main\people_animal.mp4'  # Update to video path if needed
C:\Users\Bobby\AppData\Local\Temp\ipykernel_15136\2079917165.py:16: SyntaxWarning: invalid escape sequence '\i'
  out = cv2.VideoWriter('F:\insighteye\datacenter\peopleNanimal.mp4', cv2.VideoWriter_fourcc(*'XVID'), 30, (frame_width, frame_


0: 384x640 14 persons, 7 cars, 1 motorcycle, 4 trucks, 2 elephants, 844.5ms
Speed: 3.0ms preprocess, 844.5ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 14 persons, 8 cars, 2 motorcycles, 3 trucks, 2 elephants, 798.0ms
Speed: 4.0ms preprocess, 798.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 persons, 7 cars, 2 motorcycles, 2 trucks, 2 elephants, 697.8ms
Speed: 3.0ms preprocess, 697.8ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 persons, 10 cars, 2 motorcycles, 2 trucks, 3 elephants, 721.2ms
Speed: 2.0ms preprocess, 721.2ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 14 persons, 8 cars, 2 motorcycles, 3 trucks, 3 elephants, 727.7ms
Speed: 3.0ms preprocess, 727.7ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 14 persons, 9 cars, 2 motorcycles, 3 trucks, 3 elephants, 719.8ms
Speed: 2.0ms preprocess, 719.8ms inference